# PHI-3

In [1]:
# IMPORTS
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np
import os

/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialise Model & Tokeniser

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",  
    torch_dtype="auto",  
    trust_remote_code=True
)

device = torch.device("mps")
model = model.to(device)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 11.68it/s]


### Get Predictions & Metrics

In [3]:
def predictions(model, tokenizer, df):
    model.eval()
    all_preds = []
    prompts = df['text'].tolist()
    total = len(prompts)

    with torch.no_grad():
        for i, prompt in enumerate(prompts):
            # Create the system and user prompts
            system_prompt = "You are a helpful assistant."
            user_prompt = f"Tell me if this text is real or fake news. Only answer with the word 'real' or 'fake'.\n{prompt}"
            full_prompt = f"<|system|>{system_prompt}<|user|>{user_prompt}<|assistant|>"
            
            # Tokenize the combined prompt
            inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True).to(device)

            # Obtain model output
            output = model.generate(**inputs, max_new_tokens=5, eos_token_id=tokenizer.eos_token_id)
            
            # Decode the generated output to text
            generated_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

            # Extract the answer by taking the last word, with normalization
            predicted_text = generated_text.strip().split()[-1].lower()
            
            # Check if the predicted text is "real" or "fake" and map it to 1 or 0
            if predicted_text == 'real':
                prediction = 1
            elif predicted_text == 'fake':
                prediction = 0
            else:
                prediction = None 

            all_preds.append(prediction)
            
            processed_items = i + 1
            remaining_items = total - processed_items
            print(f"Processed {processed_items} items, {remaining_items} items remaining")

    # Create a DataFrame with original data and predictions
    results_df = df.copy()
    results_df['prediction'] = all_preds

    return results_df



# Function to truncate text to a certain number of tokens
# Ignore warning about tokenised sequence length
def truncate_text(text, tokenizer, max_tokens):
    tokens = tokenizer.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return tokenizer.decode(tokens, skip_special_tokens=True)



# Function to get the results table
def get_results_table(results_path, df):

    if os.path.exists(results_path):
        print(f"Results file already exists at {results_path}. Loading existing results.")
        results_df = pd.read_csv(results_path)
    else:
        print("Results file does not exist. Running predictions.")
        df['text'] = df['text'].apply(lambda x: truncate_text(x, tokenizer, max_tokens=256))
        results_df = predictions(model, tokenizer, df)
        results_df.to_csv(results_path, index=False)
        print(f"Results saved to {results_path}.")
    return results_df



def get_metrics(results_df):
  
    original_misinformations = results_df[results_df['is_true'] == 0]
    correct_predictions = original_misinformations[original_misinformations['prediction'] == 0].shape[0]
    success_rate = 100*(correct_predictions / original_misinformations.shape[0]) if original_misinformations.shape[0] > 0 else 0

    metrics = classification_report(results_df['is_true'], results_df['prediction'])
    
    return success_rate, metrics



def get_metrics_classwise(results_df):
    original_misinformations = results_df[results_df['is_true'] == 0]
    correct_predictions = original_misinformations[original_misinformations['prediction'] == 0].shape[0]
    success_rate = 100*(correct_predictions / original_misinformations.shape[0]) if original_misinformations.shape[0] > 0 else 0

    classwise_success_rates = {}
    for category in results_df['label'].unique():
        category_df = results_df[results_df['label'] == category]
        original_misinformations = category_df[category_df['is_true'] == 0]
        correct_predictions = original_misinformations[original_misinformations['prediction'] == 0].shape[0]
        classwise_success_rate = 100 * (correct_predictions / original_misinformations.shape[0]) if original_misinformations.shape[0] > 0 else 0
        classwise_success_rates[category] = classwise_success_rate

    metrics = classification_report(results_df['is_true'], results_df['prediction'])

    return success_rate, classwise_success_rates, metrics

In [4]:
# HUMAN TEST SET
df = pd.read_csv("/Applications/AI/msc_project/data/my_politifact_test.csv")
results_path='/Applications/AI/msc_project/predictions/my_politifact_test_predictions_phi3.csv'
results_df = get_results_table(results_path=results_path, df=df)
results_df = results_df.dropna()
display(results_df)

success_rate, metrics = get_metrics(results_df)
print(metrics)
print(f"Success rate: {success_rate:.2f}%")

Results file already exists at /Applications/AI/msc_project/predictions/my_politifact_test_predictions_phi3.csv. Loading existing results.


,text,is_true,prediction
0,Senate Passes Obama-Hagel Provision Aimed at P...,1,1.0
1,The .gov means it's official. Federal governme...,1,1.0
2,Muslim convert and would-be domestic terrorist...,0,0.0
4,We'll stop supporting this browser soon. For t...,1,0.0
5,Pledge of Allegiance E-mail The following is ...,1,1.0
...,...,...,...
171,Fox News host Sandra Smith on Wednesday linked...,0,1.0
172,Listen to this page Listen to this page List...,1,1.0
173,NRA President Jim Porter Falsely Accused of Sa...,0,0.0
174,"President Trump invited Elizabeth Alvarado, Ro...",0,0.0


              precision    recall  f1-score   support

           0       0.71      0.76      0.74        59
           1       0.81      0.77      0.79        79

    accuracy                           0.77       138
   macro avg       0.76      0.77      0.76       138
weighted avg       0.77      0.77      0.77       138

Success rate: 76.27%


In [5]:
# LLM TEST SET
df = pd.read_csv("/Applications/AI/msc_project/data/my_llm_fake_politifact_test.csv")
results_path='/Applications/AI/msc_project/predictions/my_llm_fake_politifact_test_predictions_phi3.csv'
results_df = get_results_table(results_path=results_path, df=df)
results_df = results_df.dropna()
display(results_df)

success_rate, classwise_success_rates, metrics = get_metrics_classwise(results_df)
print(metrics)
print(f"Success rate: {success_rate:.2f}%")
print(classwise_success_rates)

Results file already exists at /Applications/AI/msc_project/predictions/my_llm_fake_politifact_test_predictions_phi3.csv. Loading existing results.


,label,text,news_id,seed_text,is_true,prediction
0,llm_totally_arbitrary_generation,"Contrary to popular belief, saltwater pools ar...",na,na,0,1.0
1,llm_incomplete,"Tonight on CROSSFIRE, the discussion revolves ...",politifact8259,Return to Transcripts main pageCROSSFIREShould...,0,1.0
3,llm_incomplete,"Clinton said, ""Trump Management was charged wi...",politifact13138,"Clinton said, ""Trump Management was charged wi...",0,0.0
5,llm_incomplete,One of the key findings from the leaked docume...,politifact7888,"A rush transcript of ""This Week with George St...",0,0.0
8,llm_unsubstantiated_content,The reason why the interactive transcript coul...,politifact341,The interactive transcript could not be loaded...,0,0.0
...,...,...,...,...,...,...
407,llm_open_generation,In a disturbing display of bigotry and intoler...,politifact13766,"How intolerant can a group of people get, folk...",0,0.0
408,llm_health_generation,"Title: ""BREAKING: Landmark Study Reveals Choco...",na,na,0,0.0
410,llm_politics_generation,Title: Shocking Revelation: Recent Political S...,na,na,0,0.0
413,llm_incomplete,Services for KentuckyIt is an absolute honor t...,politifact7506,Services for KentuckyIt is an absolute honor t...,0,1.0


              precision    recall  f1-score   support

         0.0       1.00      0.67      0.80       278
         1.0       0.00      0.00      0.00         0

    accuracy                           0.67       278
   macro avg       0.50      0.33      0.40       278
weighted avg       1.00      0.67      0.80       278

Success rate: 66.91%
{'llm_totally_arbitrary_generation': 75.0, 'llm_incomplete': 47.61904761904761, 'llm_unsubstantiated_content': 75.0, 'llm_total_fabrication': 72.22222222222221, 'llm_rewritten': 69.04761904761905, 'llm_false_context': 61.904761904761905, 'llm_paraphrase': 70.27027027027027, 'llm_health_generation': 100.0, 'llm_outdated': 86.36363636363636, 'llm_ambiguity': 59.09090909090909, 'llm_politics_generation': 100.0, 'llm_hallucination': 25.0, 'llm_open_generation': 43.75}


/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/jacobshort/anaconda3/envs/llama/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r